In [55]:
import numpy as np
import pandas as pd
import matplotlib as plt
import nltk
import re
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('movies.csv')

In [5]:
data.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN


In [7]:
data.shape

(9999, 9)

In [9]:
data.describe()

,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [13]:
# Calculate missing values
missing_values = data.isnull().sum()

# Calculate percentage of missing values
missing_percentage = (missing_values / len(data)) * 100

# Combine results into a DataFrame
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})

In [15]:
missing_data

,Missing Values,Percentage (%)
MOVIES,0,0.000000
YEAR,644,6.440644
GENRE,80,0.800080
RATING,1820,18.201820
ONE-LINE,0,0.000000
STARS,0,0.000000
VOTES,1820,18.201820
RunTime,2958,29.582958
Gross,9539,95.399540


In [17]:
data.drop(['Gross'],axis=1,inplace=True)

In [19]:
data['YEAR']

0            (2021)
1          (2021– )
2       (2010–2022)
3          (2013– )
4            (2021)
           ...     
9994       (2021– )
9995       (2021– )
9996       (2022– )
9997       (2021– )
9998       (2021– )
Name: YEAR, Length: 9999, dtype: object

In [13]:
data['YEAR'] = data['YEAR'].str.replace('(','')
data['YEAR'] = data['YEAR'].str.replace(')','')
data['YEAR'] = data['YEAR'].str[:4]
data['YEAR'] = data['YEAR'].str.extract(r'(\b\d{4}\b)', expand=False)

In [15]:
data['YEAR'].unique()

array(['2021', '2010', '2013', '2020', '2006', '2019', '2016', '2011',
       '2005', '2008', '2017', '1994', '2014', '2015', '2003', '2009',
       nan, '2018', '1987', '2012', '2007', '2000', '1993', '1999',
       '2001', '1997', '1989', '1975', '1995', '1984', '1998', '1966',
       '1990', '2002', '1976', '1978', '2022', '1982', '1968', '2004',
       '1996', '1971', '1980', '1962', '1991', '1960', '1988', '1969',
       '1961', '1979', '1956', '1983', '1986', '1967', '1974', '1992',
       '1958', '1932', '1941', '1950', '1946', '1981', '1952', '1957',
       '1954', '1955', '1948', '1947', '1977', '2023', '1945', '1953',
       '1985', '1973', '1972', '1965', '1944', '1933', '1938'],
      dtype=object)

In [43]:
data.shape

(9999, 8)

In [7]:
data['GENRE'].replace('\n','')

0           \nAction, Horror, Thriller            
1       \nAnimation, Action, Adventure            
2            \nDrama, Horror, Thriller            
3       \nAnimation, Adventure, Comedy            
4              \nAction, Crime, Horror            
                           ...                    
9994       \nAdventure, Drama, Fantasy            
9995    \nAnimation, Action, Adventure            
9996              \nDocumentary, Sport            
9997       \nAdventure, Drama, Fantasy            
9998       \nAdventure, Drama, Fantasy            
Name: GENRE, Length: 9999, dtype: object

In [21]:
data['GENRE'] = data['GENRE'].str.replace(r'[\n\t\r]', '', regex=True)
data['MOVIES'] = data['MOVIES'].str.replace(r'[\n\t\r]', '', regex=True)
data['ONE-LINE'] = data['ONE-LINE'].str.replace(r'[\n\t\r]', '', regex=True)
data['STARS'] = data['STARS'].str.replace(r'[\n\t\r]', '', regex=True)

In [27]:
data.describe()


,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      8970 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [31]:
data.drop(['Gross'], axis=1, inplace=True)

In [35]:
data.drop(['RunTime'], axis=1, inplace=True)

In [37]:
data.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES
0,Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,"21,062"
1,Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...","17,870"
2,The Walking Dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805"
3,Rick and Morty,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...","414,849"
4,Army of Thieves,2021,"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer| Stars:...,NaN


In [41]:
data.isnull().sum()

MOVIES         0
YEAR        1029
GENRE         80
RATING      1820
ONE-LINE       0
STARS          0
VOTES       1820
dtype: int64

In [59]:
data = data.dropna()

In [45]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

<>:7: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:7: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
C:\Users\sivas\AppData\Local\Temp\ipykernel_48984\20213847.py:7: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\sivas\AppData\Local\Temp\ipykernel_48984\20213847.py:8: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\sivas\AppData\Local\Temp\ipykernel_48984\20213847.py:12: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sivas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [47]:
data["MOVIES"] = data["MOVIES"].apply(clean)

In [49]:
data.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES
0,blood red sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,"21,062"
1,master univers revel,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...","17,870"
2,walk dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805"
3,rick morti,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...","414,849"
4,armi thiev,2021,"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer| Stars:...,NaN


In [61]:
feature = data["GENRE"].tolist()

# Create an instance of TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english")

# Fit and transform the vectorizer on our corpus
tfidf_matrix = tfidf.fit_transform(feature)

# Compute the cosine similarity matrix
similarity = cosine_similarity(tfidf_matrix)

In [63]:
similarity

array([[1.        , 0.23027265, 0.85164978, ..., 0.        , 0.23027265,
        0.        ],
       [0.23027265, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.85164978, 0.        , 1.        , ..., 0.21414531, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.21414531, ..., 1.        , 0.        ,
        0.76614339],
       [0.23027265, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.76614339, 0.        ,
        1.        ]])

In [81]:
data.head(10)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES
0,blood red sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,"21,062"
1,master univers revel,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...","17,870"
2,walk dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805"
3,rick morti,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...","414,849"
5,outer bank,2020,"Action, Crime, Drama",7.6,A group of teenagers from the wrong side of th...,"Stars:Chase Stokes, Madelyn Cl...","25,858"
6,last letter lover,2021,"Drama, Romance",6.8,A pair of interwoven stories set in the past a...,Director:Augustine Frizzell| Stars:Sha...,"5,283"
7,dexter,2006,"Crime, Drama, Mystery",8.6,"By day, mild-mannered Dexter is a blood-spatte...","Stars:Michael C. Hall, Jennife...","665,387"
8,never ever,2020,Comedy,7.9,The complicated life of a modern-day first gen...,"Stars:Maitreyi Ramakrishnan, P...","34,530"
9,virgin river,2019,"Drama, Romance",7.4,"Seeking a fresh start, nurse practitioner Meli...","Stars:Alexandra Breckenridge, ...","27,279"
10,gunpowd milkshak,2021,"Action, Adventure, Thriller",6.0,Three generations of women fight back against ...,Director:Navot Papushado| Stars:Karen ...,"17,989"


In [69]:
indices = pd.Series(data.index, 
                    index=data['MOVIES']).drop_duplicates()

In [75]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return data['MOVIES'].iloc[movieindices]

In [97]:
print(netFlix_recommendation("walk dead"))

['walk dead', 'american horror stori', 'gerald game', 'eliza grave', 'hous', 'red dot', 'kadav', 'tabula rasa', 'il legam', 'ghost lab']


In [107]:
data[data['MOVIES'].isin(list(netFlix_recommendation("walk dead")))]

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES
2,walk dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805"
14,american horror stori,2011,"Drama, Horror, Thriller",8.0,An anthology series centering on different cha...,"Stars:Lady Gaga, Kathy Bates, ...","286,488"
452,gerald game,2017,"Drama, Horror, Thriller",6.5,A couple tries to spice up their marriage in a...,Director:Mike Flanagan| Stars:Carla Gu...,"95,660"
523,eliza grave,2014,"Drama, Horror, Thriller",6.8,An Oxford graduate takes up a job in a mental ...,Director:Brad Anderson| Stars:Kate Bec...,"53,438"
533,hous,2020,"Drama, Horror, Thriller",6.5,A refugee couple makes a harrowing escape from...,Director:Remi Weekes| Stars:Sope Diris...,"32,061"
1032,red dot,2021,"Drama, Horror, Thriller",5.5,"When Nadja becomes pregnant, they make an atte...",Director:Alain Darborg| Stars:Nanna Bl...,"11,555"
1271,kadav,2020,"Drama, Horror, Thriller",5.1,In the starving aftermath of a nuclear disaste...,Director:Jarand Herdal| Stars:Gitte Wi...,"8,104"
1537,tabula rasa,2017,"Drama, Horror, Thriller",8.0,A young woman with amnesia becomes a key figur...,"Stars:Veerle Baetens, Stijn Va...","9,128"
1742,il legam,2020,"Drama, Horror, Thriller",4.7,While visiting her fiancé's mother in southern...,Director:Domenico Emanuele de Feudis| ...,"3,162"
1850,ghost lab,2021,"Drama, Horror, Thriller",5.2,"After witnessing a haunting in their hospital,...",Director:Paween Purijitpanya| Stars:Th...,973


In [105]:
list(netFlix_recommendation("walk dead"))

['walk dead',
 'american horror stori',
 'gerald game',
 'eliza grave',
 'hous',
 'red dot',
 'kadav',
 'tabula rasa',
 'il legam',
 'ghost lab']

In [115]:
data[data['GENRE'].isin(['Drama, Horror, Thriller            '])]

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES
2,walk dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...","885,805"
14,american horror stori,2011,"Drama, Horror, Thriller",8.0,An anthology series centering on different cha...,"Stars:Lady Gaga, Kathy Bates, ...","286,488"
452,gerald game,2017,"Drama, Horror, Thriller",6.5,A couple tries to spice up their marriage in a...,Director:Mike Flanagan| Stars:Carla Gu...,"95,660"
523,eliza grave,2014,"Drama, Horror, Thriller",6.8,An Oxford graduate takes up a job in a mental ...,Director:Brad Anderson| Stars:Kate Bec...,"53,438"
533,hous,2020,"Drama, Horror, Thriller",6.5,A refugee couple makes a harrowing escape from...,Director:Remi Weekes| Stars:Sope Diris...,"32,061"
1032,red dot,2021,"Drama, Horror, Thriller",5.5,"When Nadja becomes pregnant, they make an atte...",Director:Alain Darborg| Stars:Nanna Bl...,"11,555"
1271,kadav,2020,"Drama, Horror, Thriller",5.1,In the starving aftermath of a nuclear disaste...,Director:Jarand Herdal| Stars:Gitte Wi...,"8,104"
1537,tabula rasa,2017,"Drama, Horror, Thriller",8.0,A young woman with amnesia becomes a key figur...,"Stars:Veerle Baetens, Stijn Va...","9,128"
1742,il legam,2020,"Drama, Horror, Thriller",4.7,While visiting her fiancé's mother in southern...,Director:Domenico Emanuele de Feudis| ...,"3,162"
1850,ghost lab,2021,"Drama, Horror, Thriller",5.2,"After witnessing a haunting in their hospital,...",Director:Paween Purijitpanya| Stars:Th...,973
